# Download the OCRd text for ALL the digitised periodicals in Trove!

Finding which periodical issues in Trove have OCRd text you can download is not as easy as it should be. The `fullTextInd` index doesn't seem to distinguish between digitised works (with OCR) and born-digital publications (like PDFs) without downloadable text. You can use `has:correctabletext` to find articles with OCR, but you can't get a full list of the periodicals the articles come from using the `title` facet. As [this notebook explains](Create-digitised-journals-list.ipynb), you can search for `nla.obj`, but this returns both digitised works and publications supplied through edeposit. In previous harvests of OCRd text I processed all of the titles returned by the `nla.obj` search, finding out whether there was any OCRd text by just requesting it and seeing what came back. But the number of non-digitised works on the list of periodicals in digital form has skyrocketed through the edeposit scheme and this approach is no longer practical. It just means you waste a lot of time asking for things that don't exist.

For the latest harvest I took a different approach. I only processed periodicals in digital form that *weren't* identified as coming through edeposit. These are the publications with a `fulltext_url_type` value of either 'digitised' or 'other' in my [dataset of digital periodicals](digital-journals-20220831.csv). Is it possible that there's some downloadable text in edeposit works that's now missing from the harvest? Yep, but I think this is a much more sensible, straightforward, and reproduceable approach.

That's not the only problem. As I noted when creating the list of periodicals in digital form, there are duplicates in the list, so they have to be removed. You then have to find information obout the issues available for each title. This is not provided by the Trove API, but there is an internal API used in the web interface that you can access – see [this notebook for details](Get-text-from-a-Trove-journal.ipynb). I also noticed that sometimes where there's a single issue of a title, it's presented as if each page is an issue. I think I've found a work around for that as well.

All these doubts, inconsistencies and workarounds mean that I'm fairly certain I don't have *everything*. But I do think I have *most* of the OCRd text available from digitised periodicals, and I do have a methodology, documented in this notebook, that at least provides a starting point for further investigation. As I noted in my comments on the plan for a Trove Researcher Platform, it would be great if more metadata for digitised works, other than newspapers, was made available through the API.

So this notebook puts together the [list of periodicals](digital-journals-20220831.csv) [created by this notebook](Create-digitised-journals-list.ipynb) with the [code in this notebook](Get-text-from-a-Trove-journal.ipynb), and downloads as much OCRd text from digitised periodicals in the `journals` zone as it can find. If you're going to try this, you'll need a lots of patience and lots of disk space. Needless to say, don't try this on a cloud service like Binder.

Fortunately you don't have to do it yourself, as I've already run the harvest and made all the text files available. See below for details.

I repeat, **you probably don't want to do this yourself**. The point of this notebook is really to document the methodology used to create the repository.

If you really, really do want to do it yourself, you should first [generate an updated list of digitised periodicals](Create-digitised-journals-list.ipynb).

## Here's a harvest I prepared earlier...

I last ran this harvest in August 2022. Here are the results:

* 1,430 periodicals had OCRd text available for download
* OCRd text was downloaded from 41,645 periodical issues 
* About 9gb of text was downloaded

Note that, the previous harvest excluded periodicals with the format 'government publication', this time I've kept them in, even though there's overlap with the 'Book' zone. I'm planning a new database to bring together the book and periodical harvests anyway. Also the harvested collection last time included some details of periodicals without OCRd text, but they've all now been excluded. So the full collection on CloudStor *only* includes periodicals with OCRd text. This makes it more compact and manageable. Finally, I think some duplicates slipped into the last harvest – I hope I've removed them this time.

The list of digital journals with OCRd text is available both as [human-readable list](digital-journals-with-text.md) and a [CSV formatted spreadsheet](digital-journals-with-text-20220831.csv).

The complete collection of text files for all the journals can be downloaded [from this repository on CloudStor](https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h).

## Setting things up

In [13]:
# Let's import the libraries we need.
import json
import os
import re
import time
from datetime import datetime

import arrow
import pandas as pd
import requests
import requests_cache
from bs4 import BeautifulSoup
from IPython.display import HTML, FileLink, display
from requests.adapters import HTTPAdapter
from requests.exceptions import ConnectionError, Timeout
from requests.packages.urllib3.util.retry import Retry
from slugify import slugify
from tqdm.auto import tqdm

s = requests_cache.CachedSession()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
s.mount("https://", HTTPAdapter(max_retries=retries))
s.mount("http://", HTTPAdapter(max_retries=retries))

In [60]:
def get_work_metadata(obj_id):
    """
    Extracts metadata embedded as a JSON string in a work's HTML page.
    See: https://glam-workbench.net/trove-books/metadata-for-digital-works/
    """
    # Get the HTML page
    response = requests.get(f"https://nla.gov.au/{obj_id}")
    # Search for the JSON string using regex
    try:
        work_data = re.search(
            r"var work = JSON\.parse\(JSON\.stringify\((\{.*\})", response.text
        ).group(1)
    except AttributeError:
        # Just in case it's not there...
        work_data = "{}"
        print("No data found!")
    # Return the JSON data
    return json.loads(work_data)


def harvest_metadata(obj_id):
    """
    This calls an internal API from a journal landing page to extract a list of available issues.
    """
    start_url = "https://nla.gov.au/{}/browse?startIdx={}&rows=20&op=c"
    # The initial startIdx value
    start = 0
    # Number of results per page
    n = 20
    issues = []
    with tqdm(desc="Issues", leave=False) as pbar:
        # If there aren't 20 results on the page then we've reached the end, so continue harvesting until that happens.
        while n == 20:
            # Get the browse page
            response = s.get(start_url.format(obj_id, start), timeout=60)
            # Beautifulsoup turns the HTML into an easily navigable structure
            soup = BeautifulSoup(response.text, "lxml")
            # Find all the divs containing issue details and loop through them
            details = soup.find_all(class_="l-item-info")
            for detail in details:
                issue = {}
                title = detail.find("h3")
                if title:
                    issue["title"] = title.text
                    issue["id"] = title.parent["href"].strip("/")
                else:
                    issue["title"] = "No title"
                    issue["id"] = detail.find("a")["href"].strip("/")
                try:
                    # Get the issue details
                    issue["details"] = detail.find(
                        class_="obj-reference content"
                    ).string.strip()
                except (AttributeError, IndexError):
                    issue["details"] = "issue"
                # Get the number of pages
                try:
                    issue["pages"] = int(
                        re.search(
                            r"^(\d+)",
                            detail.find("a", attrs={"data-pid": issue["id"]}).text,
                            flags=re.MULTILINE,
                        ).group(1)
                    )
                except AttributeError:
                    # A number of 'periodicals' are actually single issues in which pages are split and treated as individual issues, they don't have a number of pages
                    # Doens't seem to be an easy way of capturing these
                    issue["pages"] = 0
                issues.append(issue)
                # print(issue)
                if not response.from_cache:
                    time.sleep(0.5)
            # Increment the startIdx
            start += n
            # Set n to the number of results on the current page
            n = len(details)
            pbar.update(n)
    return issues


def issues_have_pages(issues):
    for issue in issues:
        if issue["pages"] > 0:
            return True
    return False


def download_issue(issue_id, last_page):
    url = f"https://trove.nla.gov.au/{issue_id}/download?downloadOption=ocr&firstPage=0&lastPage={last_page}"
    # print(url)
    # Get the file
    try:
        r = s.get(url, timeout=180)
    except (Timeout, ConnectionError) as err:
        print(f"{type(err).__name__}: {url}")
    else:
        # Check there was no error
        if r.status_code == requests.codes.ok:
            # Check that the file's not empty
            r.encoding = "utf-8"
            # Check that the file isn't HTML (some not found pages don't return 404s)
            # BS is too lax and will pass text files that happen to have html tags in them
            # if BeautifulSoup(r.text, "html.parser").find("html") is None:
            if (
                len(r.text) > 0
                and not r.text.isspace()
                and not re.search(r"</HTML>", r.text, re.IGNORECASE)
            ):
                return r


def save_ocr(issues, obj_id, title=None, output_dir="journals"):
    """
    Download the OCRd text for each issue.
    """
    processed_issues = []
    if not title:
        title = issues[0]["title"]
    output_path = os.path.join(output_dir, "{}-{}".format(slugify(title)[:50], obj_id))
    texts_path = os.path.join(output_path, "texts")
    os.makedirs(texts_path, exist_ok=True)

    if issues_have_pages(issues) is False:
        # Some things that look like collections of issues are actually just a single issue
        # Let's try and get text from the parent id
        work_data = get_work_metadata(obj_id)
        try:
            issue_pages = len(work_data["children"]["page"])
        except KeyError:
            processed_issues = issues
        else:
            last_page = issue_pages - 1
            r = download_issue(obj_id, last_page)
            if r:
                file_name = f"{slugify(title)[:50]}-{obj_id}.txt"
                file_path = os.path.join(texts_path, file_name)
                with open(file_path, "w", encoding="utf-8") as text_file:
                    text_file.write(r.text)
                issue = {
                    "title": title,
                    "id": obj_id,
                    "details": "",
                    "pages": issue_pages,
                    "text_file": file_name,
                }
                if not r.from_cache:
                    time.sleep(1)
                processed_issues.append(issue)
            else:
                processed_issues = issues
    else:
        for issue in tqdm(issues, desc="Texts", leave=False):
            # Default values
            issue["text_file"] = ""
            if issue["pages"] != 0:
                # print(book['title'])
                # The index value for the last page of an issue will be the total pages - 1
                last_page = issue["pages"] - 1
                file_name = "{}-{}-{}.txt".format(
                    slugify(issue["title"])[:50],
                    slugify(issue["details"])[:50],
                    issue["id"],
                )
                file_path = os.path.join(texts_path, file_name)
                # Check to see if the file has already been harvested
                if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
                    # print('Already saved')
                    issue["text_file"] = file_name
                else:
                    r = download_issue(issue["id"], last_page)
                    # In case the page number is wrong, try going back a few
                    while r is None and last_page > max(0, issue["pages"] - 5):
                        last_page = last_page - 1
                        r = download_issue(issue["id"], last_page)
                    if r:
                        with open(file_path, "w", encoding="utf-8") as text_file:
                            text_file.write(r.text)
                        issue["text_file"] = file_name
                        if not r.from_cache:
                            time.sleep(1)
            processed_issues.append(issue)
    df = pd.DataFrame(processed_issues)
    # Remove empty directories
    """
    try:
        os.rmdir(texts_path)
        os.rmdir(output_path)
    except OSError:
        #It's not empty, so add list of issues
        df.to_csv(os.path.join(output_path, '{}-issues.csv'.format(obj_id)), index=False)
    """
    try:
        os.rmdir(texts_path)
    except OSError:
        pass
    df.to_csv(os.path.join(output_path, "{}-issues.csv".format(obj_id)), index=False)

## Process all the journals!

As already mentioned, this takes a long time. It will also probably fail at various points and you'll have to run it again. If you do restart, the script will start at the beginning, but won't redownload any text files have already been harvested.

Results for each journal are saved in a separate directory in the outpur directory (which defaults to `journals`). The name of the journal directory is created using the journal title and journal id. Inside this directory is a CSV formatted file containing details of all the available issues, and a `texts` sub-directory to contain the downloaded text files.

The individual file names are created using the journal title, issue details, and issue identifier. So the resulting hierarchy might look something like this:

```
journals
    - angry-penguins-nla.obj-320790312
        - nla.obj-320790312-issues.csv
        - texts
            - angry-penguins-broadsheet-no-1-nla.obj-320791009.txt
```

The CSV list of issues includes the following fields:

* `details` – string with issue details, might include dates, issue numbers etc.
* `id` – issue identifier
* `pages` – number of pages in this issue
* `text_file` – file name of any downloaded OCRd text
* `title` – journal title (if not supplied this will be extracted from issue browse list and so might differ from original journal title)

Note that if the `text_file` field is empty, it means that no OCRd text could be extracted for that particular issue. Note also that if no OCRd text is available, no `texts` directory will be created.

In [61]:
# You can provide a different output_dir if you want
def process_titles(output_dir="journals"):
    df = pd.read_csv("digital-journals-20220831.csv")
    # df = pd.read_csv('government-publications-periodicals.csv')
    # Filter out edeposit titles, and drop any duplicates
    journals = (
        df.loc[df["fulltext_url_type"] != "edeposit"]
        .sort_values(by=["trove_id", "fulltext_url_type"])
        .drop_duplicates(subset="trove_id", keep="first")
        .to_dict("records")
    )
    for journal in tqdm(journals, desc="Journals"):
        issues = harvest_metadata(journal["trove_id"])
        if issues:
            save_ocr(
                issues,
                journal["trove_id"],
                title=journal["title"],
                output_dir=output_dir,
            )
        else:
            print(journal["trove_id"])

In [ ]:
# Start harvesting!!!!
process_titles("/home/tim/Workspace/workingdata/Trove/journals/")

## Gather data about the harvest

Because the harvesting takes a long time and is prone to failure, it seemed wise to gather data at the end, rather than keeping a running total.

The cells below create a list of journals that have OCRd text. The list has the following fields:

* `fulltext_url` – the url of the landing page of the digital version of the journal
* `title` – the title of the journal
* `trove_id` – the 'nla.obj' part of the fulltext_url, a unique identifier for the digital journal
* `trove_url` – url of the journal's metadata record in Trove
* `issues` – the number of available issues
* `issues_with_text` – the number of issues that OCRd text could be downloaded from
* `directory` – the directory in which the files from this journal have been saved (relative to the output directory)


In [4]:
def collect_issue_data(output_path="journals"):
    titles_with_text = []
    df = pd.read_csv("digital-journals-20220831.csv", keep_default_na=False)
    # df = pd.read_csv('government-publications-periodicals-20210802.csv')
    journals = (
        df.loc[df["fulltext_url_type"] != "edeposit"]
        .sort_values(by=["trove_id", "fulltext_url_type"])
        .drop_duplicates(subset="trove_id", keep="first")
        .to_dict("records")
    )
    for j in journals:
        j_dir = os.path.join(
            output_path, "{}-{}".format(slugify(j["title"])[:50], j["trove_id"])
        )
        if os.path.exists(j_dir):
            csv_file = os.path.join(j_dir, "{}-issues.csv".format(j["trove_id"]))
            try:
                issues_df = pd.read_csv(csv_file, keep_default_na=False)
            except pd.errors.EmptyDataError:
                print(j_dir)

            try:
                num_text = issues_df.loc[issues_df["text_file"] != ""].shape[0]
            except KeyError:
                pass
            else:
                if num_text > 0:
                    j["issues_with_text"] = num_text
                    j["issues"] = issues_df.shape[0]
                    j["directory"] = "{}-{}".format(
                        slugify(j["title"])[:50], j["trove_id"]
                    )
                    titles_with_text.append(j)
    return titles_with_text

In [5]:
# Gather the data
# titles_with_text = collect_issue_data()
titles_with_text = collect_issue_data("/home/tim/Workspace/workingdata/Trove/journals/")

Convert to a dataframe.

In [16]:
try:
    df_text = pd.DataFrame(titles_with_text)
except NameError:
    df_text = pd.read_csv("digital-journals-with-text-20220831.csv")
df_text.head()

,title,contributor,issued,format,fulltext_url,trove_url,trove_id,fulltext_url_type,issues_with_text,issues,directory
0,"Science and technology statement (Canberra, A....","Australia. Department of Industry, Technology ...",1900-2022,"Periodical | Periodical/Journal, magazine, oth...",https://nla.gov.au/nla.obj-1006986858,https://trove.nla.gov.au/work/5555883,nla.obj-1006986858,digitised,6,6,science-and-technology-statement-canberra-a-c-...
1,Research report / Australian National University,Australian National University,1990-2022,"Periodical | Periodical/Journal, magazine, other",https://nla.gov.au/nla.obj-1018350073,https://trove.nla.gov.au/work/8712197,nla.obj-1018350073,digitised,2,2,research-report-australian-national-university...
2,Airline activities of Ansett Transport Industr...,Ansett Transport Industries,1900-1973,"Periodical | Periodical/Journal, magazine, other",https://nla.gov.au/nla.obj-1036685302,https://trove.nla.gov.au/work/34691176,nla.obj-1036685302,digitised,1,1,airline-activities-of-ansett-transport-industr...
3,Anzac bulletin : issued to members of the Aust...,Australia. High Commission (Great Britain),1916-1919,"Periodical | Periodical/Journal, magazine, other",http://nla.gov.au/nla.obj-1037567,https://trove.nla.gov.au/work/12653430,nla.obj-1037567,digitised,205,205,anzac-bulletin-issued-to-members-of-the-austra...
4,The 23rd : the voice of the battalion,"Australia. Army. Battalion, 23rd",1917-1919,"Periodical | Periodical/Journal, magazine, other",http://nla.gov.au/nla.obj-10414986,https://trove.nla.gov.au/work/31637350,nla.obj-10414986,digitised,30,30,the-23rd-the-voice-of-the-battalion-nla.obj-10...


Save as a CSV file.

Let's have a peek inside...

In [7]:
# Number of journals with OCRd text
df_text.shape

(1430, 11)

In [8]:
# Total number of issues
df_text["issues"].sum()

41904

In [9]:
# Number of issues with OCRd text
df_text["issues_with_text"].sum()

41645

In [14]:
csv_file = f'digital-journals-with-text-{datetime.now().strftime("%Y%m%d")}.csv'
df_text.to_csv(csv_file, index=False)
display(HTML(f'<a href="{csv_file}" download="{csv_file}">{csv_file}</a>'))

## Create a markdown-formatted list

In [ ]:
df_text.sort_values(by=["title"], inplace=True)
with open("digital-journals-with-text.md", "w") as md_file:
    md_file.write("# Digitised journals from Trove with OCRd text")
    md_file.write(
        f'\n\nThis harvest was completed on {arrow.now("Australia/Canberra").format("D MMMM YYYY")}.'
    )
    md_file.write(f"\n\nNumber of journals harvested: {df_text.shape[0]:,}")
    md_file.write(
        f'\n\nNumber of issues with OCRd text: {df_text["issues_with_text"].sum():,}'
    )
    md_file.write("\n\n----\n\n")
    for row in df_text.itertuples():
        md_file.write(f"\n### {row.title}")
        if row.contributor:
            md_file.write(f"\n**{row.contributor}, {row.issued}**")
        else:
            md_file.write(f"\n**{row.issued}**")
        md_file.write(f"  \n{row.format}")
        md_file.write(
            f"\n\n{row.issues_with_text} of {row.issues} issues have OCRd text available for download."
        )
        md_file.write(f"\n\n* [Details on Trove]({row.trove_url})\n")
        md_file.write(f"* [Browse issues on Trove]({row.fulltext_url})\n")
        md_file.write(
            f"* [Download issue data as CSV from CloudStor](https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h/download?path=%2F{row.directory}&files={row.trove_id}-issues.csv)\n"
        )
        md_file.write(
            f"* [Download all OCRd text from CloudStor](https://cloudstor.aarnet.edu.au/plus/s/QOmnqpGQCNCSC2h/download?path=%2F{row.directory})\n"
        )

display(FileLink("digital-journals-with-text.md"))

----

Created by [Tim Sherratt](https://timsherratt.org/) for the [GLAM Workbench](https://glam-workbench.github.io/).

Work on this notebook was supported by the [Humanities, Arts and Social Sciences (HASS) Data Enhanced Virtual Lab](https://tinker.edu.au/).